# Assignment 2 - Fish Prices

---
This notebook shows all the step used to answer questions of assignment 2 from [Econometrics](https://edu.epfl.ch/coursebook/fr/econometrics-FIN-403) courses.

**Author**: [Paulo](https://people.epfl.ch/paulo.ribeirodecarvalho)

---

## Import Libraries

In [7]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

## Load data

In [8]:
file_path = "fish.csv"
fish = pd.read_csv(file_path)
display(fish)

,avgprc,lavgprc,totqty,ltotqty,speed2,speed3,wave2,wave3,mon,tues,wed,thurs,t
0,0.700286,-0.356266,4080,8.313852,15,20,7.5,9.0,1,0,0,0,1
1,1.005336,0.005322,3466,8.150757,10,20,5.0,7.5,0,0,1,0,2
2,1.393918,0.332118,2295,7.738488,10,20,6.0,4.0,0,0,0,1,3
3,1.775487,0.574075,1870,7.533694,15,20,6.0,5.0,0,0,0,0,4
4,0.826799,-0.190194,6885,8.837100,10,20,3.5,3.5,1,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,0.429408,-0.845347,3450,8.146130,15,15,4.5,3.5,1,0,0,0,93
93,0.907724,-0.096815,1230,7.114770,12,25,4.5,4.0,0,1,0,0,94
94,1.238571,0.213958,1250,7.130899,12,25,5.0,4.5,0,0,1,0,95
95,1.250000,0.223144,3980,8.289037,15,18,6.0,4.5,0,0,0,1,96


## Part 1

For the first few questions, we use the variable `lavgprc` (= the log of the average price for fish) as the dependent variable.

### a)

In [9]:
# Define the variables
y = fish['lavgprc']  # Dependent variable
X = fish[['mon', 'tues', 'wed', 'thurs', 't']]  # Independent variables

# Add a constant term for the intercept
X = sm.add_constant(X)

# Perform the regression
model = sm.OLS(y, X).fit()

# Display the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                lavgprc   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     1.701
Date:                Tue, 21 Nov 2023   Prob (F-statistic):              0.142
Time:                        13:38:33   Log-Likelihood:                -45.022
No. Observations:                  97   AIC:                             102.0
Df Residuals:                      91   BIC:                             117.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0730      0.115     -0.633      0.5

***Answer:*** Statistically no evidence that price varies within a week. Indeed, p-values of week-day variables are all bigger than the 5% significance level.

### b)

In [10]:
# Define the variables
y = fish['lavgprc']  # Dependent variable
X = fish[['mon', 'tues', 'wed', 'thurs', 't', 'wave2', 'wave3']]  # Independent variables

# Add a constant term for the intercept
X = sm.add_constant(X)

# Perform the regression
model_with_wave = sm.OLS(y, X).fit()

# Display the regression results
print(model_with_wave.summary())

                            OLS Regression Results                            
Dep. Variable:                lavgprc   R-squared:                       0.309
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     5.699
Date:                Tue, 21 Nov 2023   Prob (F-statistic):           1.84e-05
Time:                        13:38:33   Log-Likelihood:                -31.396
No. Observations:                  97   AIC:                             78.79
Df Residuals:                      89   BIC:                             99.39
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9203      0.190     -4.848      0.0

In [11]:
# Let's see if wave2 and wave3 are jointly significant
f_test = model_with_wave.compare_f_test(model)

# Print the results
print(f"F-statistic : {f_test[0]:.3f}")
print(f"p-value : {f_test[1]:.3f}")

F-statistic : 14.436
p-value : 0.000


***Answer:*** From previous results, we see that variable `wave2` and `wave3` are individually and jointly statically significant at a $5\%$ significance level.

### c)